                        llama3-8b  pass@1: 0.38 | phi3 pass@1: 0.60  [HumanEval]

In [1]:
from data import CodeCompletionGenerator, load_config
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm as tqdm


config_name = "phi_3_instruct"
code_gen = CodeCompletionGenerator.from_config(config_name, device="mps")
# Specific Indices & System Prompts for HumanEval cases performance
code_gen.run_human_eval_test(indices = [3], global_system_prompt="Do not fuck it up")

#### Fast Adaptation

In [5]:
# Reflective Adaptation | Use Groq for fast iteration | CUDA device could also be fast (?)
import groq
from human_eval.data import read_problems
problems = read_problems()
problem = problems['HumanEval/0']
prompt = problem['prompt']
# print(prompt)




from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

